# Lab 4: Redis Exercises - Key-Value Store

## 🎯 Objectives
- Understand Redis data structures
- Implement caching strategies
- Practice session management
- Learn rate limiting techniques
- Optimize performance with Redis

## 📋 Prerequisites
- Complete Lab 1 (Setup & Connections)
- Redis container is running


In [1]:
import redis
import json
import time
from datetime import datetime, timedelta

# Redis connection
redis_client = redis.Redis(host='localhost', port=6379, db=0, decode_responses=True)

# Test connection
redis_client.ping()
print("✅ Redis connected successfully!")
print(f"Redis version: {redis_client.info('server')['redis_version']}")


✅ Redis connected successfully!
Redis version: 7.2.10


## Exercise 1: Basic Operations


In [2]:
# Basic string operations
redis_client.set("user:001", "John Doe")
redis_client.setex("session:001", 3600, "active")  # Expires in 1 hour

# Get values
user = redis_client.get("user:001")
session = redis_client.get("session:001")
ttl = redis_client.ttl("session:001")

print(f"User: {user}")
print(f"Session: {session}")
print(f"Session TTL: {ttl} seconds")

# Hash operations
redis_client.hset("product:001", mapping={
    "name": "iPhone 15 Pro",
    "price": "25000000",
    "category": "electronics"
})

product = redis_client.hgetall("product:001")
print(f"Product: {product}")

# List operations
redis_client.lpush("recent_products", "prod_001", "prod_002", "prod_003")
recent = redis_client.lrange("recent_products", 0, -1)
print(f"Recent products: {recent}")

# Set operations
redis_client.sadd("category:electronics", "prod_001", "prod_002", "prod_003")
electronics = redis_client.smembers("category:electronics")
print(f"Electronics products: {electronics}")

print("✅ Basic operations completed!")


User: John Doe
Session: active
Session TTL: 3600 seconds
Product: {'name': 'iPhone 15 Pro', 'price': '25000000', 'category': 'electronics'}
Recent products: ['prod_003', 'prod_002', 'prod_001']
Electronics products: {'prod_002', 'prod_001', 'prod_003'}
✅ Basic operations completed!


## Exercise 2: Caching Strategy


In [3]:
class CacheService:
    def __init__(self, redis_client):
        self.redis = redis_client
    
    def cache_product(self, product_id, product_data, ttl=1800):
        """Cache product data with TTL"""
        key = f"product:{product_id}"
        self.redis.setex(key, ttl, json.dumps(product_data))
        return True
    
    def get_cached_product(self, product_id):
        """Get cached product data"""
        key = f"product:{product_id}"
        data = self.redis.get(key)
        return json.loads(data) if data else None
    
    def cache_user_session(self, user_id, session_data, ttl=3600):
        """Cache user session"""
        key = f"session:{user_id}"
        self.redis.setex(key, ttl, json.dumps(session_data))
        return True
    
    def invalidate_cache(self, pattern):
        """Invalidate cache by pattern"""
        keys = self.redis.keys(pattern)
        if keys:
            return self.redis.delete(*keys)
        return 0

# Initialize cache service
cache_service = CacheService(redis_client)

# Cache sample product
product_data = {
    "productId": "prod_001",
    "name": "iPhone 15 Pro",
    "price": 25000000,
    "category": "electronics"
}

cache_service.cache_product("prod_001", product_data)
cached_product = cache_service.get_cached_product("prod_001")
print(f"Cached product: {cached_product}")

# Cache user session
session_data = {
    "userId": "user_001",
    "cart": ["prod_001", "prod_002"],
    "lastActivity": datetime.utcnow().isoformat()
}

cache_service.cache_user_session("user_001", session_data)
cached_session = cache_service.get_cached_product("user_001")  # This will be None
print(f"Cached session exists: {redis_client.exists('session:user_001')}")

print("✅ Caching strategy implemented!")


Cached product: {'productId': 'prod_001', 'name': 'iPhone 15 Pro', 'price': 25000000, 'category': 'electronics'}
Cached session exists: 1
✅ Caching strategy implemented!


## Exercise 3: Rate Limiting


In [4]:
class RateLimiter:
    def __init__(self, redis_client):
        self.redis = redis_client
    
    def is_allowed(self, user_id, limit=100, window=3600):
        """Check if user is within rate limit"""
        key = f"rate_limit:{user_id}"
        current = self.redis.incr(key)
        
        if current == 1:
            self.redis.expire(key, window)
        
        return current <= limit
    
    def get_remaining_requests(self, user_id, limit=100):
        """Get remaining requests for user"""
        key = f"rate_limit:{user_id}"
        current = self.redis.get(key)
        if current is None:
            return limit
        return max(0, limit - int(current))

# Initialize rate limiter
rate_limiter = RateLimiter(redis_client)

# Test rate limiting
user_id = "user_001"
limit = 5

print(f"Testing rate limiting for user {user_id} (limit: {limit})")

for i in range(7):
    allowed = rate_limiter.is_allowed(user_id, limit, 60)  # 60 second window
    remaining = rate_limiter.get_remaining_requests(user_id, limit)
    print(f"Request {i+1}: {'✅ Allowed' if allowed else '❌ Blocked'} (Remaining: {remaining})")

print("✅ Rate limiting implemented!")


Testing rate limiting for user user_001 (limit: 5)
Request 1: ✅ Allowed (Remaining: 4)
Request 2: ✅ Allowed (Remaining: 3)
Request 3: ✅ Allowed (Remaining: 2)
Request 4: ✅ Allowed (Remaining: 1)
Request 5: ✅ Allowed (Remaining: 0)
Request 6: ❌ Blocked (Remaining: 0)
Request 7: ❌ Blocked (Remaining: 0)
✅ Rate limiting implemented!


## Exercise 4: Performance Benchmarking


In [5]:
# Performance benchmarking
def benchmark_redis_operations():
    """Benchmark different Redis operations"""
    operations = {
        "SET": lambda: redis_client.set("test:key", "test_value"),
        "GET": lambda: redis_client.get("test:key"),
        "HSET": lambda: redis_client.hset("test:hash", "field", "value"),
        "HGET": lambda: redis_client.hget("test:hash", "field"),
        "LPUSH": lambda: redis_client.lpush("test:list", "item"),
        "LRANGE": lambda: redis_client.lrange("test:list", 0, -1),
        "SADD": lambda: redis_client.sadd("test:set", "member"),
        "SMEMBERS": lambda: redis_client.smembers("test:set")
    }
    
    results = {}
    
    for op_name, op_func in operations.items():
        start_time = time.time()
        for _ in range(1000):  # Run 1000 operations
            op_func()
        end_time = time.time()
        
        duration = end_time - start_time
        ops_per_sec = 1000 / duration
        results[op_name] = {
            "duration": duration,
            "ops_per_sec": ops_per_sec
        }
    
    return results

# Run benchmark
print("🚀 Running Redis performance benchmark...")
benchmark_results = benchmark_redis_operations()

# Display results
print("\n📊 Performance Results:")
print("-" * 50)
for op, metrics in benchmark_results.items():
    print(f"{op:8} | {metrics['ops_per_sec']:8.0f} ops/sec | {metrics['duration']:.4f}s")

# Cleanup test keys
redis_client.delete("test:key", "test:hash", "test:list", "test:set")

print("\n✅ Performance benchmarking completed!")


🚀 Running Redis performance benchmark...

📊 Performance Results:
--------------------------------------------------
SET      |     5415 ops/sec | 0.1847s
GET      |     8115 ops/sec | 0.1232s
HSET     |     7236 ops/sec | 0.1382s
HGET     |     7857 ops/sec | 0.1273s
LPUSH    |     7241 ops/sec | 0.1381s
LRANGE   |      860 ops/sec | 1.1633s
SADD     |     6485 ops/sec | 0.1542s
SMEMBERS |     6654 ops/sec | 0.1503s

✅ Performance benchmarking completed!


## 🎓 Student Exercises - Advanced Redis Practice

### Exercise 5: Async Redis Queue Implementation
**Objective**: Build asynchronous task queue system using Redis

**Scenario**: You need to build a **Background Job Processing System** for an e-commerce platform with:
- Order processing queue
- Email notification queue
- Image processing queue
- Analytics data processing queue

**Tasks**:
1. **Implement async Redis queue**:
   - Create job queue using Redis Lists
   - Implement job producer and consumer
   - Add job priority handling
   - Implement job retry mechanism

2. **Build queue management features**:
   - Job status tracking
   - Queue monitoring dashboard
   - Failed job handling
   - Queue statistics

3. **Implement advanced queue patterns**:
   - Delayed job execution
   - Job scheduling
   - Dead letter queue
   - Job batching

**Requirements**:
- Use Redis Lists for queue implementation
- Implement proper error handling
- Add job serialization/deserialization
- Include monitoring and logging
- Handle concurrent job processing

---

### Exercise 6: Real-time Analytics with Redis Streams
**Objective**: Build real-time analytics system using Redis Streams

**Scenario**: Build a **Real-time E-commerce Analytics Platform** with:
- Live user activity tracking
- Real-time sales analytics
- Live inventory monitoring
- Real-time recommendation updates

**Tasks**:
1. **Implement Redis Streams**:
   - Create event streams for user activities
   - Implement stream consumers
   - Add stream processing logic
   - Handle stream persistence

2. **Build analytics features**:
   - Real-time user behavior tracking
   - Live sales dashboard
   - Inventory alerts
   - Performance metrics

3. **Create monitoring system**:
   - Stream health monitoring
   - Consumer lag tracking
   - Performance metrics
   - Alert system

**Requirements**:
- Use Redis Streams for event processing
- Implement proper stream management
- Add real-time data visualization
- Handle high-volume data streams
- Include fault tolerance

---

### Exercise 7: Distributed Caching and Session Management
**Objective**: Implement distributed caching and session management system

**Scenario**: Build a **Scalable Session Management System** for a multi-server e-commerce platform with:
- Distributed session storage
- Session replication
- Cache invalidation strategies
- User preference caching

**Tasks**:
1. **Implement distributed session management**:
   - Multi-server session sharing
   - Session replication strategies
   - Session failover handling
   - Session cleanup automation

2. **Build advanced caching strategies**:
   - Cache-aside pattern
   - Write-through caching
   - Write-behind caching
   - Cache warming strategies

3. **Create cache management tools**:
   - Cache invalidation system
   - Cache performance monitoring
   - Cache hit/miss analytics
   - Cache optimization tools

**Requirements**:
- Use Redis Cluster for distributed caching
- Implement proper session management
- Add cache invalidation strategies
- Include performance monitoring
- Handle cache consistency

---

### Exercise 8: Redis Pub/Sub and Event-Driven Architecture
**Objective**: Build event-driven architecture using Redis Pub/Sub

**Scenario**: Build an **Event-Driven E-commerce System** with:
- Real-time notifications
- Event-driven microservices
- Real-time inventory updates
- Live order status updates

**Tasks**:
1. **Implement Redis Pub/Sub**:
   - Create event publishers
   - Implement event subscribers
   - Add event filtering
   - Handle message persistence

2. **Build event-driven features**:
   - Real-time notifications
   - Inventory synchronization
   - Order status updates
   - User activity broadcasting

3. **Create event management system**:
   - Event routing
   - Event filtering
   - Event persistence
   - Event replay capability

**Requirements**:
- Use Redis Pub/Sub for messaging
- Implement proper event handling
- Add event persistence
- Include error handling
- Handle message ordering

---

### Exercise 9: Redis Performance Optimization and Monitoring
**Objective**: Optimize Redis performance and implement comprehensive monitoring

**Tasks**:
1. **Implement performance optimization**:
   - Memory optimization strategies
   - Connection pooling
   - Pipeline operations
   - Lua scripting for complex operations

2. **Build monitoring system**:
   - Performance metrics collection
   - Memory usage monitoring
   - Connection monitoring
   - Query performance analysis

3. **Create optimization tools**:
   - Performance benchmarking
   - Memory usage analysis
   - Query optimization
   - Capacity planning

**Requirements**:
- Use Redis performance tuning techniques
- Implement comprehensive monitoring
- Add performance benchmarking
- Include capacity planning
- Handle performance bottlenecks

---

### Exercise 10: Redis Security and Data Protection
**Objective**: Implement security measures and data protection for Redis

**Scenario**: Secure a **Production Redis System** with:
- Data encryption
- Access control
- Audit logging
- Data backup and recovery

**Tasks**:
1. **Implement security measures**:
   - Redis AUTH configuration
   - ACL (Access Control Lists)
   - Network security
   - Data encryption

2. **Build data protection features**:
   - Data backup strategies
   - Disaster recovery
   - Data retention policies
   - Compliance monitoring

3. **Create security monitoring**:
   - Access logging
   - Security alerts
   - Audit trails
   - Compliance reporting

**Requirements**:
- Implement Redis security best practices
- Add comprehensive logging
- Include backup and recovery
- Handle security compliance
- Monitor security events

---

## 📝 Submission Guidelines

### For Each Exercise:
1. **Complete all tasks** as specified
2. **Include proper documentation** and comments
3. **Test your code** with sample data
4. **Handle errors** appropriately
5. **Optimize performance** where possible

### Code Quality Requirements:
- Follow Python best practices
- Use meaningful variable names
- Include type hints where appropriate
- Write clean, readable code
- Add comprehensive comments

### Testing Requirements:
- Test with various Redis scenarios
- Test edge cases and error conditions
- Verify expected outputs
- Include performance benchmarks

### Documentation Requirements:
- Explain your Redis usage patterns
- Document performance optimizations
- Include usage examples
- Provide performance analysis

---

## 🏆 Bonus Challenges

### Challenge 1: Redis Cluster Implementation
Implement a Redis Cluster setup with automatic failover and data sharding.

### Challenge 2: Redis with Machine Learning
Use Redis for real-time machine learning model serving and feature store.

### Challenge 3: Redis as a Database
Build a complete application using Redis as the primary database with proper data modeling.

### Challenge 4: Redis Integration with Cloud Services
Integrate Redis with cloud services like AWS ElastiCache, Azure Cache, or Google Cloud Memorystore.

---

**Good luck with your Redis journey! 🚀**


In [ ]:
# Exercise 5: Async Redis Queue Implementation - Starter Code
# Build asynchronous task queue system using Redis

import asyncio
import json
import time
from datetime import datetime
from typing import Dict, Any, Optional

class AsyncRedisQueue:
    def __init__(self, redis_client, queue_name: str = "default"):
        self.redis = redis_client
        self.queue_name = queue_name
        self.processing_queue = f"{queue_name}:processing"
        self.failed_queue = f"{queue_name}:failed"
    
    def enqueue_job(self, job_data: Dict[str, Any], priority: int = 0) -> str:
        """
        Add a job to the queue
        TODO: Implement this function
        """
        pass
    
    def dequeue_job(self, timeout: int = 0) -> Optional[Dict[str, Any]]:
        """
        Get a job from the queue
        TODO: Implement this function
        """
        pass
    
    def mark_job_completed(self, job_id: str) -> bool:
        """
        Mark a job as completed
        TODO: Implement this function
        """
        pass
    
    def mark_job_failed(self, job_id: str, error: str) -> bool:
        """
        Mark a job as failed
        TODO: Implement this function
        """
        pass
    
    def get_queue_stats(self) -> Dict[str, Any]:
        """
        Get queue statistics
        TODO: Implement this function
        """
        pass

class JobProcessor:
    def __init__(self, queue: AsyncRedisQueue):
        self.queue = queue
        self.running = False
    
    async def start_processing(self):
        """
        Start processing jobs from the queue
        TODO: Implement this function
        """
        pass
    
    async def process_job(self, job_data: Dict[str, Any]):
        """
        Process a single job
        TODO: Implement this function
        """
        pass
    
    def stop_processing(self):
        """
        Stop processing jobs
        TODO: Implement this function
        """
        pass

# Sample job data structure (for reference)
sample_jobs = [
    {
        "id": "job_001",
        "type": "email_notification",
        "data": {
            "to": "user@example.com",
            "subject": "Order Confirmation",
            "body": "Your order has been confirmed"
        },
        "priority": 1,
        "created_at": datetime.now().isoformat()
    },
    {
        "id": "job_002", 
        "type": "image_processing",
        "data": {
            "image_url": "https://example.com/image.jpg",
            "operations": ["resize", "compress"]
        },
        "priority": 2,
        "created_at": datetime.now().isoformat()
    }
]

# Initialize queue and processor
queue = AsyncRedisQueue(redis_client, "ecommerce_jobs")
processor = JobProcessor(queue)

print("Exercise 5: Async Redis Queue Implementation")
print("Complete the AsyncRedisQueue and JobProcessor methods above!")


In [ ]:
# Exercise 6: Real-time Analytics with Redis Streams - Starter Code
# Build real-time analytics system using Redis Streams

class RedisStreamAnalytics:
    def __init__(self, redis_client):
        self.redis = redis_client
        self.stream_name = "user_activity"
        self.consumer_group = "analytics_group"
    
    def add_event(self, event_type: str, user_id: str, data: Dict[str, Any]) -> str:
        """
        Add an event to the stream
        TODO: Implement this function
        """
        pass
    
    def create_consumer_group(self, group_name: str, start_id: str = "0") -> bool:
        """
        Create a consumer group for the stream
        TODO: Implement this function
        """
        pass
    
    def read_events(self, consumer_name: str, count: int = 10) -> List[Dict[str, Any]]:
        """
        Read events from the stream
        TODO: Implement this function
        """
        pass
    
    def process_user_activity(self, events: List[Dict[str, Any]]) -> Dict[str, Any]:
        """
        Process user activity events for analytics
        TODO: Implement this function
        """
        pass
    
    def get_stream_info(self) -> Dict[str, Any]:
        """
        Get stream information and statistics
        TODO: Implement this function
        """
        pass

class RealTimeAnalytics:
    def __init__(self, stream_analytics: RedisStreamAnalytics):
        self.stream_analytics = stream_analytics
        self.metrics = {}
    
    def track_user_action(self, user_id: str, action: str, metadata: Dict[str, Any]):
        """
        Track user action in real-time
        TODO: Implement this function
        """
        pass
    
    def get_real_time_metrics(self) -> Dict[str, Any]:
        """
        Get real-time analytics metrics
        TODO: Implement this function
        """
        pass
    
    def generate_alerts(self, threshold: float) -> List[Dict[str, Any]]:
        """
        Generate alerts based on metrics
        TODO: Implement this function
        """
        pass

# Sample event data structure (for reference)
sample_events = [
    {
        "event_type": "page_view",
        "user_id": "user_001",
        "data": {
            "page": "/products/iphone",
            "timestamp": datetime.now().isoformat(),
            "session_id": "session_001"
        }
    },
    {
        "event_type": "purchase",
        "user_id": "user_001", 
        "data": {
            "product_id": "prod_001",
            "amount": 25000000,
            "timestamp": datetime.now().isoformat()
        }
    }
]

# Initialize analytics system
stream_analytics = RedisStreamAnalytics(redis_client)
real_time_analytics = RealTimeAnalytics(stream_analytics)

print("Exercise 6: Real-time Analytics with Redis Streams")
print("Complete the RedisStreamAnalytics and RealTimeAnalytics methods above!")


In [ ]:
# Exercise 7: Distributed Caching and Session Management - Starter Code
# Implement distributed caching and session management system

class DistributedSessionManager:
    def __init__(self, redis_client):
        self.redis = redis_client
        self.session_prefix = "session:"
        self.user_prefix = "user:"
    
    def create_session(self, user_id: str, session_data: Dict[str, Any], ttl: int = 3600) -> str:
        """
        Create a new user session
        TODO: Implement this function
        """
        pass
    
    def get_session(self, session_id: str) -> Optional[Dict[str, Any]]:
        """
        Get session data by session ID
        TODO: Implement this function
        """
        pass
    
    def update_session(self, session_id: str, data: Dict[str, Any]) -> bool:
        """
        Update session data
        TODO: Implement this function
        """
        pass
    
    def invalidate_session(self, session_id: str) -> bool:
        """
        Invalidate a session
        TODO: Implement this function
        """
        pass
    
    def get_user_sessions(self, user_id: str) -> List[Dict[str, Any]]:
        """
        Get all active sessions for a user
        TODO: Implement this function
        """
        pass

class AdvancedCacheManager:
    def __init__(self, redis_client):
        self.redis = redis_client
        self.cache_prefix = "cache:"
    
    def cache_set(self, key: str, value: Any, ttl: int = 3600, strategy: str = "cache_aside") -> bool:
        """
        Set cache with different strategies
        TODO: Implement this function
        """
        pass
    
    def cache_get(self, key: str) -> Optional[Any]:
        """
        Get cache value
        TODO: Implement this function
        """
        pass
    
    def cache_invalidate_pattern(self, pattern: str) -> int:
        """
        Invalidate cache by pattern
        TODO: Implement this function
        """
        pass
    
    def cache_warm_up(self, keys: List[str], data_source_func) -> Dict[str, Any]:
        """
        Warm up cache with data
        TODO: Implement this function
        """
        pass
    
    def get_cache_stats(self) -> Dict[str, Any]:
        """
        Get cache statistics
        TODO: Implement this function
        """
        pass

# Sample session and cache data structures (for reference)
sample_session_data = {
    "user_id": "user_001",
    "login_time": datetime.now().isoformat(),
    "last_activity": datetime.now().isoformat(),
    "ip_address": "192.168.1.100",
    "user_agent": "Mozilla/5.0...",
    "preferences": {
        "theme": "dark",
        "language": "en",
        "notifications": True
    }
}

sample_cache_data = {
    "user_profile": {
        "user_id": "user_001",
        "name": "John Doe",
        "email": "john@example.com",
        "preferences": {"theme": "dark"}
    },
    "product_catalog": {
        "category": "electronics",
        "products": ["prod_001", "prod_002", "prod_003"]
    }
}

# Initialize managers
session_manager = DistributedSessionManager(redis_client)
cache_manager = AdvancedCacheManager(redis_client)

print("Exercise 7: Distributed Caching and Session Management")
print("Complete the DistributedSessionManager and AdvancedCacheManager methods above!")
